In [ ]:
if get_prox == False
    state = 0

if get_prox == True
    state = 1

In [ ]:
# Main -> state machine

obstacle = False

while(1):
    if shortest_path() == False:
        state = 0


    # robot run    
    elif shortest_path() == True:
        state = 1
        
    obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
            
            
        
    

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    